# ✉️ Messages
  <img src="https://github.com/niladridutta1988/wf_agentic_ai_training/blob/main/Batch2/lca-langchainV1-essentials/python_colab/assets/LC_Messages.png?raw=1" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [24]:
!pip install -qU langchain-groq langgraph langchain-community

In [25]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

## Human👨‍💻 and AI 🤖 Messages

In [26]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq

""" agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a full-stack comedian"
) """

# Initialize the Groq model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)

# Use the llm object in the agent creation
agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian"
)

In [27]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [28]:
print(result["messages"][-1].content)

I'm doing great, thanks for asking. I just got back from a gig where I had to explain to an audience member that my jokes weren't actually "stacked" like a full-stack developer, but rather, I'm a comedian who can do it all – write, act, and even code. I mean, who needs a 401k when you can have a 404 error, right? (laughs)


In [29]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [30]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: I'm doing great, thanks for asking. I just got back from a gig where I had to explain to an audience member that my jokes weren't actually "stacked" like a full-stack developer, but rather, I'm a comedian who can do it all – write, act, and even code. I mean, who needs a 401k when you can have a 404 error, right? (laughs)



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

In [31]:
agent = create_agent(
    # model="openai:gpt-5-nano",
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [32]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond's sacred ground,
Nine innings spun, a tale unbound.
The crack of bat, a symphony sweet,
A home run's roar, the crowd's wild beat.

The pitcher winds, a tale of might,
A fastball's zip, a curve's subtle flight.
The catcher crouches, a guardian true,
A strike called out, the umpire's cue.

The shortstop snags, a line drive's fate,
A double play, the infield's eager state.
The outfielder leaps, a fly ball's grasp,
A catch made sure, the crowd's joyful clasp.

The game of strategy, a dance of skill,
A test of wills, where heroes stand still.
The roar of the crowd, a deafening sound,
As baseball's magic, forever spins around.


#### Dictionaries

In [33]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Racing, swift as wind
Golden shoes on burning track
Speed's fierce, fleeting dance


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages.

In [34]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [35]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [36]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush


In [37]:
result["messages"][-1].content

'Snowflakes gently fall\nBlanketing the winter scene\nFrosty peaceful hush'

In [38]:
print(len(result["messages"]))

4


In [39]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (16g7yq6wg)
 Call ID: 16g7yq6wg
  Args:
    text: Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [40]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='cdae1165-ce77-4285-9f09-f267a82275f6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '16g7yq6wg', 'function': {'arguments': '{"text":"Snowflakes gently fall\\nBlanketing the winter scene\\nFrosty peaceful hush"}', 'name': 'check_haiku_lines'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 280, 'total_tokens': 333, 'completion_time': 0.068041679, 'completion_tokens_details': None, 'prompt_time': 0.038225575, 'prompt_tokens_details': None, 'queue_time': 0.006132625, 'total_time': 0.106267254}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_020e283281', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb305-8c52-7f40-92d1-832bc414ac88-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Snowflakes gently fall\nB

You can select just the last message, and you can see where the final message is coming from.

In [41]:
result["messages"][-1]

AIMessage(content='Snowflakes gently fall\nBlanketing the winter scene\nFrosty peaceful hush', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 335, 'total_tokens': 355, 'completion_time': 0.022582857, 'completion_tokens_details': None, 'prompt_time': 0.139961652, 'prompt_tokens_details': None, 'queue_time': 0.071895404, 'total_time': 0.162544509}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6c980774ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb305-8d02-7870-b638-83a9b47ab192-0', usage_metadata={'input_tokens': 335, 'output_tokens': 20, 'total_tokens': 355})

In [42]:
result["messages"][-1].usage_metadata

{'input_tokens': 335, 'output_tokens': 20, 'total_tokens': 355}

In [43]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 20,
  'prompt_tokens': 335,
  'total_tokens': 355,
  'completion_time': 0.022582857,
  'completion_tokens_details': None,
  'prompt_time': 0.139961652,
  'prompt_tokens_details': None,
  'queue_time': 0.071895404,
  'total_time': 0.162544509},
 'model_name': 'llama-3.1-8b-instant',
 'system_fingerprint': 'fp_6c980774ec',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None,
 'model_provider': 'groq'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [44]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [45]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (16g7yq6wg)
 Call ID: 16g7yq6wg
  Args:
    text: Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Snowflakes gently fall
Blanketing the winter scene
Frosty peaceful hush
